In [27]:
import os
from collections import defaultdict
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from dotenv import load_dotenv

import import_ipynb
from util_make_datasets import make_datasets
from util_load_best_params_svd import load_best_params_svd

# Подготовка данных

In [206]:
load_dotenv()

data_path = os.path.abspath(os.getenv('data_path'))
params_path = im.params_path = os.path.abspath(os.getenv('params_path'))
matrix_folder = os.path.abspath(os.getenv('matrix_path'))
matrix_name = os.getenv('matrix_name')

df_ratings, df_books, df_tags, df_book_tags, df_users = make_datasets(data_path)
best_params = load_best_params_svd(params_path, df_ratings, mode='increment')

# Метрики

In [207]:
def precision_at_k(true_books, predicted_books, k=5):
    correct_predictions = len(set(true_books).intersection(set(predicted_books[:k])))
    precision = correct_predictions / min(k, len(predicted_books))
    
    return precision

def recall_at_k(true_books, predicted_books, k=5):
    correct_predictions = len(set(true_books).intersection(set(predicted_books[:k])))
    recall = correct_predictions / len(true_books)
    
    return recall 

def ndcg_at_k(tb, pb, k=5):
    dcg = 0
    idcg = 0
    for i, book_id in enumerate(tb):
        if book_id in pb:
            relevance = tb[book_id]
            dcg += (2 ** relevance - 1) / np.log2(i + 2)
        idcg += (2 ** 5 - 1) / np.log2(i + 2)

    ndcg = dcg / idcg if idcg != 0 else 0
    
    return ndcg

In [208]:
# Словарь хранения метрик
metrics = {
    'SVD':{'precision': [], 'recall': [], 'ndcg': []},
    'popularity':{'precision': [], 'recall': [], 'ndcg': []},
    'combined_hybrid':{'precision': [], 'recall': [], 'ndcg': []},
}

# Отдельные функции

## SVD

In [209]:
def get_recommendations_svd(user_id, N=5):
    """Матричная факторизация (SVD).
Функция возвращает топ-N книг с наибольшим предсказанным рейтингом для заданного пользователя"""
        # Выполним предсказание для всех книг и отсортируем их в порядке убывания оценки
    predictions = [model.predict(user_id, item) for item in items_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)
        
    top_books_svd = [pred.iid for pred in predictions[:N]]
        
    return top_books_svd

In [210]:
metrics_svd = {'precision': [], 'recall': [], 'ndcg': []}

## Popularity

In [211]:
def get_popularity_recommendation_ids(df_ratings, N=5):
    """Модель популярности (топ-N популярных книг).
Функция показывает топ-N самых популярных книг по количеству оценок."""
    # Получаем топ-N самых популярных книг
    popular_books = df_ratings['book_id'].value_counts().index[:N]

    popular_book_ids = list(popular_books)

    # Возвращаем id топ-N книг
    return popular_book_ids

In [212]:
metrics_popularity = {'precision': [], 'recall': [], 'ndcg': []}

In [205]:
# Загрузим данные в формат, подходящий для scikit-surprise
reader = Reader(rating_scale=(1, 5))  # Устанавливаем диапазон рейтинга
data = Dataset.load_from_df(df_ratings[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.01, random_state=42)

# Рассчитаем гиперпараметры
best_params = load_best_params_svd(params_path, df_ratings, 'increment')
n_factors, n_epochs, lr_all, reg_all = best_params.values()

# Обучим модель SVD
model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)
model.fit(trainset)

# Получим множество книг для предсказания
items_to_predict = set(i[1] for i in testset)

In [215]:
# Предсказания SVD
predicted_books_svd = defaultdict(list)
for user_id in set(u for u,i,r in testset):
    recs = get_recommendations_svd(user_id, N=5)
    predicted_books_svd[user_id] = recs

In [216]:
# Предсказания popularity
predicted_books = get_popularity_recommendation_ids(df_ratings, N=5)

In [217]:
# Реальные предпочтения
true_books = defaultdict(set)
true_rating = defaultdict(dict)
for u,i,r in testset:
    true_books[u].add(i)
    true_rating[u][i] = r

## Рассчет метрик

In [218]:
for user_id in set(u for u,i,r in testset):
    precision = precision_at_k(true_books[user_id], predicted_books_svd[user_id], k=5)
    recall = recall_at_k(true_books[user_id], predicted_books_svd[user_id], k=5)
    ndcg = ndcg_at_k(true_rating[user_id], predicted_books_svd[user_id], k=5)

    metrics_svd['precision'].append(precision)
    metrics_svd['recall'].append(recall)
    metrics_svd['ndcg'].append(ndcg)

    precision = precision_at_k(true_books[user_id], predicted_books, k=5)
    recall = recall_at_k(true_books[user_id], predicted_books, k=5)
    ndcg = ndcg_at_k(true_rating[user_id], predicted_books, k=5)

    metrics_popularity['precision'].append(precision)
    metrics_popularity['recall'].append(recall)
    metrics_popularity['ndcg'].append(ndcg)

In [219]:
metrics['SVD']['precision'] = np.mean(metrics_svd['precision'])
metrics['SVD']['recall'] = np.mean(metrics_svd['recall'])
metrics['SVD']['ndcg'] = np.mean(metrics_svd['ndcg'])

In [220]:
metrics['popularity']['precision'] = np.mean(metrics_popularity['precision'])
metrics['popularity']['recall'] = np.mean(metrics_popularity['recall'])
metrics['popularity']['ndcg'] = np.mean(metrics_popularity['ndcg'])

# Гибридная модель

In [223]:
def get_hybrid_recommendation(user_id, df_ratings, weights=None):
    # Определим веса для каждой из рекомендаций
    if weights is None:
        weights = [2, 3, 4, 5]
    
    # Получаем рекомендации книг по каждой модели в соответствии с весами
    popularity_rec       = get_popularity_recommendation_ids(df_ratings, N=5)
    collaborative_rec    = get_recommendations_svd(user_id, N=5)
    
    # Отбираем уникальные книги
    recommendations = (popularity_rec + collaborative_rec)
    unique_recs = list(set(recommendations))
    
    return unique_recs

In [224]:
def get_user_type(user_id, df_users, threshold=None):
    # Находим характеристики пользователя
    avg_user_rating = df_users[df_users['user_id'] == user_id]['avg_user_rating'].iloc[0]
    num_user_ratings = df_users[df_users['user_id'] == user_id]['num_user_ratings'].iloc[0]
    user_activity = df_users[df_users['user_id'] == user_id]['user_activity'].iloc[0]
    
    # Определяем погоровые значения
    if threshold is None:
        threshold = [3.5,    # Средняя оценка
                     10,     # Количество оценок
                     0.1]    # Рейтинг активности (Процентное отношение оценок к количеству книг)

    th_avg_user_rating, th_num_user_ratings, th_user_activity = threshold

    if avg_user_rating >= th_avg_user_rating and \
      (num_user_ratings >= th_num_user_ratings or user_activity >= th_user_activity):
        user_type = 'active'
    else:
        user_type = 'new'

    return user_type

In [225]:
def get_combined_recomendation_by_user_type(user_id, df_ratings, df_users):
    # Определяем тип пользователя
    user_type = get_user_type(user_id, df_users)

    # Новым пользователям предлагаются деперсонализированные популярные книги
    # Активным пользователям - гибридные взвешенные рекомендации
    if user_type == 'new':
        combined_recs = get_popularity_recommendation_ids(df_ratings, 15)
    elif user_type == 'active':
        combined_recs = get_hybrid_recommendation(user_id, df_ratings, weights=None)

    return combined_recs

In [230]:
metrics_combined_hybrid = {'precision': [], 'recall': [], 'ndcg': []}

In [226]:
# Предсказания Hybrid
predicted_books_hybrid = defaultdict(list)
for user_id in set(u for u,i,r in testset):
    recs = get_combined_recomendation_by_user_type(user_id, df_ratings, df_users)
    predicted_books_hybrid[user_id] = recs

## Расчет метрик

In [231]:
for user_id in set(u for u,i,r in testset):
    precision = precision_at_k(true_books[user_id], predicted_books_hybrid[user_id], k=5)
    recall = recall_at_k(true_books[user_id], predicted_books_hybrid[user_id], k=5)
    ndcg = ndcg_at_k(true_rating[user_id], predicted_books_hybrid[user_id], k=5)

    metrics_combined_hybrid['precision'].append(precision)
    metrics_combined_hybrid['recall'].append(recall)
    metrics_combined_hybrid['ndcg'].append(ndcg)

In [233]:
metrics['combined_hybrid']['precision'] = np.mean(metrics_svd['precision'])
metrics['combined_hybrid']['recall'] = np.mean(metrics_svd['recall'])
metrics['combined_hybrid']['ndcg'] = np.mean(metrics_svd['ndcg'])

# Таблица результатов

In [239]:
df_results = pd.DataFrame(metrics).T.rename_axis('Model').reset_index()
pd.options.display.float_format = '{:,.6f}'.format
print("\nТаблица результатов:")
print(df_results)


Таблица результатов:
             Model  precision   recall     ndcg
0              SVD   0.000103 0.000451 0.000366
1       popularity   0.000077 0.000301 0.000110
2  combined_hybrid   0.000103 0.000451 0.000366


# Выводы

Рассматривая представленные результаты, можно заметить следующее:
* Модель SVD: Показала одинаковые результаты с комбинированной моделью по всем трем метрикам.  
  Это связано с тем, что гибридная модель основана в том числе и на модели SVD
  Однако, хотя показатели низкие, модель демонстрирует хорошее соотношение точности и охвата.
* Популярные рекомендации: Этот подход прост и быстр в реализации, однако его точность ниже, чем SVD или гибридная модель.
  Причина заключается в том, что такая стратегия игнорирует личные предпочтения пользователей и ориентируется исключительно на популярность книг.
* Комбинированная модель: Демонстрирует тот же уровень точности и полноты, что и SVD, однако имеет выигрышное преимущество в возможности адаптироваться к различным сценариям запросов (что показано в модуле hybrid_system и приложении app/main.py).

Особенности работы моделей:
* Модель SVD: Отличается хорошей производительностью и точностью, однако ограниченность вычислительных мощностей и количества данных влияет на конечные результаты. При увеличении размера тестового набора показатели улучшаются.
* Популярные рекомендации: Работает чрезвычайно быстро, легко масштабируемая, но страдает низкой точностью. Такой подход полезен в ситуациях, когда важна высокая пропускная способность и минимальны требования к индивидуализации рекомендаций.
* Комбинированная модель: Может обеспечить оптимальный баланс между точностью и производительностью, позволяя выдавать рекомендации, комбинирующие несколько различных подходов, а также в зависисмости от типа пользователя.